In [230]:
import pandas as pd
import numpy as np
import plotly.express as px
import gantt
import importlib
from distinctipy import distinctipy
from datetime import datetime

In [239]:
# filename = '../ciment_quebec/solution/CQ/2020101_det.csv'
# filename = './20201003.csv'
filename = './20201126.csv'
# filename = './20201010.csv'
filename = './20201031.csv'
filename = './20231217.csv'
filename = './20231218.csv'
# filename = './20231219.csv'


In [241]:
df = pd.read_csv(filename,sep=";")
t1 = df.loc[:,['Del',	'ArrDock',	'FinDock','Driver','instance','Order','Cust','Depot'	]]
t1['Task'] = t1.apply(lambda x:f"Load_{int(x.Del)}",axis=1)
t1['Op'] = t1.apply(lambda x:f"L{int(x.Order)}",axis=1)
t1['Operation'] ="Loading"
t1.rename(columns={'ArrDock':'start','FinDock':'end'},inplace=True)
t2 = df.loc[:,['Del',	'StartDel','FinDel','Driver','instance','Order','Cust','Depot'	]]
t2['Task'] = t2.apply(lambda x:f"UnLoad_{int(x.Del)}",axis=1)
t2['Op'] = t2.apply(lambda x:f"U{int(x.Order)}",axis=1)
t2['Operation'] ="Unloading"
t2.rename(columns={'StartDel':'start','FinDel':'end'},inplace=True)
df = pd.concat([t1,t2])
importlib.reload(gantt)
df['start'] = df.apply(lambda x: gantt.time_to_date( x.instance,x.start),axis=1)
df['end'] = df.apply(lambda x: gantt.time_to_date( x.instance,x.end),axis=1)

# gantt.showGanttDel(df)
# df
import importlib
importlib.reload(gantt)
gantt.ShowLoadings(df.copy())
gantt.ShowUnloadings(df.copy())

In [232]:
df.head()

,Del,start,end,Driver,instance,Order,Cust,Depot,Task,Op,Operation
0,39,2023-12-18 10:00:00,2023-12-18 10:06:00,1,20231218,0,0,0,Load_39,L0,Loading
1,40,2023-12-18 10:24:00,2023-12-18 10:30:00,3,20231218,0,0,0,Load_40,L0,Loading
2,41,2023-12-18 10:51:00,2023-12-18 10:57:00,2,20231218,0,0,0,Load_41,L0,Loading
3,42,2023-12-18 07:36:00,2023-12-18 07:43:00,4,20231218,1,1,1,Load_42,L1,Loading
4,43,2023-12-18 07:52:00,2023-12-18 07:59:00,6,20231218,1,1,1,Load_43,L1,Loading


In [ ]:
Prism = [
    'rgb(0, 255, 0)', 'rgb(255, 0, 255)', 'rgb(0, 128, 255)',
    'rgb(255, 128, 0)', 'rgb(128, 191, 128)']  # 'rgb(74, 17, 157)',
# 'rgb(209, 2, 37)', 'rgb(74, 17, 157)', 'rgb(176, 114, 254)',
# 'rgb(3, 255, 226)', 'rgb(52, 125, 6)', 'rgb(0, 128, 128)',
# 'rgb(250, 81, 134)', 'rgb(255, 128, 0)'
# ]
df['Task'] = df.apply(lambda x: f"Depot {x.Depot+1}", axis=1)
# color = [f"rgb{distinctipy.get_rgb256(x)}" for x in distinctipy.get_colors(14)]
# print(color)
df_ordre = df.loc[df['Operation'] == "Loading"]
if type(df_ordre) == pd.core.series.Series:
    df_ordre = df_ordre.to_frame().T
df_ordre = df_ordre.sort_values(by=['Depot', 'start'])
fig = px.timeline(df_ordre, x_start="start", x_end="end", y="Task", text='Driver', width=900, height=200,
template='ggplot2',
                    # color_discrete_sequence=["red", "green", "blue", "goldenrod", "magenta"],
                    color="Order",
                #   color_continuous_scale=Prism, opacity=1
                    )
# ,color_continuous_scale=["cyan", "green", "blue", "goldenrod", "magenta","magenta"]
# )
fig.update_yaxes(
    # autorange="reversed",
    #  showgrid=True,
    #  showticklabels=False,
    #  ticks="",
)  # otherwise tasks are listed from the bottom up

# fig.update_layout(
# yaxis=dict(visible=False),  # Hide the y-axis
# xaxis=dict(type="date"),     # Use a date-based x-axis
# title="Timeline",
# font=dict(
# family="Courier New, monospace",
# size=5,  # Set the font size here
# color="RebeccaPurple"
# )
# )
# fig.update_traces(textposition='inside')
# fig.update_layout(uniformtext_minsize=4)
fig.update_layout(uniformtext_minsize=5, 
                    yaxis_title="",
    xaxis=dict(  tick0=datetime.strptime(
    "2023-12-18 06:45:00", "%Y-%m-%d %H:%M:%S"),
    range=[datetime.strptime("2023-12-18 07:00:00", "%Y-%m-%d %H:%M:%S"), datetime.strptime(
    "2023-12-18 11:40:00", "%Y-%m-%d %H:%M:%S")],
    )
                    )
# , uniformtext_mode='hide')
fig.update_coloraxes(showscale=False)
# fig.update_traces(showlegend=False)
fig.update_xaxes(
    tickformat="%H:%M",
    nticks=25,
    tickformatstops=[
        dict(dtickrange=[3600000, 86400000], value="%H:%M")]  # range is 1 hour to 24 hours
)
# for template in ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]:


fig.write_image(f"depot_loadings.pdf")
fig.write_image(f"depot_loadings.png")
fig.show()


In [108]:
Prism = [
'rgb(0, 255, 0)', 'rgb(255, 0, 255)', 'rgb(0, 128, 255)',
'rgb(255, 128, 0)', 'rgb(128, 191, 128)']  # 'rgb(74, 17, 157)',
# 'rgb(209, 2, 37)', 'rgb(74, 17, 157)', 'rgb(176, 114, 254)',
# 'rgb(3, 255, 226)', 'rgb(52, 125, 6)', 'rgb(0, 128, 128)',
# 'rgb(250, 81, 134)', 'rgb(255, 128, 0)'
# ]
depotList = np.unique(df['Depot'])
df['Task'] = df.apply(lambda x: f"Cust {x.Cust}", axis=1)
# color = [f"rgb{distinctipy.get_rgb256(x)}" for x in distinctipy.get_colors(14)]
# print(color)
df_ordre=df[df['Operation']=="Unloading"]
if type(df_ordre) == pd.core.series.Series:
    df_ordre = df_ordre.to_frame().T
df_ordre.sort_values(by=['start'], inplace=True)
fig = px.timeline(df_ordre, x_start="start", x_end="end", y="Task", text='Order', width=1000, height=600,
                    template='ggplot2',
                    # color_discrete_sequence=["red", "green", "blue", "goldenrod", "magenta"],
                    color="Driver",
                    color_continuous_scale=Prism, opacity=1
                    )
# ,color_continuous_scale=["cyan", "green", "blue", "goldenrod", "magenta","magenta"]
# )
fig.update_yaxes(autorange="reversed",
                    #  showgrid=True,
                    #  showticklabels=False,
                    #  ticks="",
                    )  # otherwise tasks are listed from the bottom up

    # fig.update_layout(
# yaxis=dict(visible=False),  # Hide the y-axis
# xaxis=dict(type="date"),     # Use a date-based x-axis
# title="Timeline",
# font=dict(
# family="Courier New, monospace",
# size=5,  # Set the font size here
# color="RebeccaPurple"
# )
# )
# fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=6)  # , uniformtext_mode='hide')
fig.update_coloraxes(showscale=False)
# fig.update_traces(showlegend=False)
fig.update_xaxes(
tickformat="%H:%M",
nticks=25,
tickformatstops=[
    dict(dtickrange=[3600000, 86400000], value="%H:%M")]  # range is 1 hour to 24 hours
)
# for template in ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]:
#     fig.update_layout(template=template, title="Mt Bruno Elevation: '%s' theme" % template)
#     fig.show()

# fig.write_image(f"depot_{dep}.pdf")
# fig.write_image(f"depot_{dep}.png")
fig.show()


/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_56847/3733511792.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [19]:
import importlib
importlib.reload(gantt)
gantt.showGanttDepot(df)

In [20]:
import importlib
importlib.reload(gantt)
gantt.show(df)

In [6]:
import importlib
importlib.reload(gantt)
gantt.showGanttDriver(df)

KeyError: 'startLoad'

In [82]:
# df = pd.read_csv(filename,sep=";")
# df['start'] = df.apply(lambda x: gantt.time_to_date( x.Date,x.ArrDock),axis=1)
# df['end'] = df.apply(lambda x: gantt.time_to_date( x.Date,x.FinDel),axis=1)
# stat1 = df.groupby(['ordreId','OrderDemand','delId','driverId','depotLoc']).agg({'delDemand':['sum'],'ArrivalTimeDel':['sum'],'early_tw':['sum']})
# stat2 = df.groupby(['driverId','depotLoc','delId','ordreId','OrderDemand']).agg({'ArrivalTimeDel':['sum'],'early_tw':['sum']})
# stat3 = df.groupby(['depotLoc','driverId','driverRank','delId','delDemand','ordreId','OrderDemand']).agg({'ArrivalTimeDel':['sum'],'early_tw':['sum']})

# stat2.to_excel('stats.xlsx','stat2')


# stat4 = df.groupby(['ordreId','OrderDemand']).agg({'delDemand':['sum']})
# stat4.columns = [x[0] for x in stat4.columns]
# stat4.reset_index(inplace=True)
# stat4['diff']= stat4['OrderDemand'] - stat4['delDemand']
# order_unserved = stat4[stat4['diff']!=0]
# if len(order_unserved) > 0:
#     order_unserved.to_csv('order_unserved.csv')
# order_unserved.head()
# print(f"{len(df['driverId'].unique())} drivers used")
# print(f"{len(df['depotLoc'].unique())} depots used")
